In [392]:
from bs4 import BeautifulSoup
import pickle
import time
import requests
import csv
import re
from dateutil import parser
from datetime import timedelta
from datetime import datetime
import random
import os
import pandas as pd
import numpy as np

In [517]:
#pickle.dump({}, open("movies.pkl", 'w'))

In [526]:
# Scrape a List of movies released in 2015 with wikipedia articles (and the links)
year = 2014
response = requests.get("https://en.wikipedia.org/wiki/%d_in_film"%year)
soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all("table", class_="wikitable")

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [527]:
movies = {}
for table in tables[2:8]: # The tables I want are 4 - 7
    for movie in table.find_all("i"):
        movie = movie.find("a")
        if movie is not None:
            url = movie.get("href").split("/")[-1].encode("utf-8")
            text = movie.text.encode("utf-8")
            movies[text] = {"url":url}

In [528]:
with open("movies.pkl", "r") as f:
    all_movies = pickle.load(f)
    all_movies.update(movies)
    movies = all_movies
    
with open("movies.pkl", "w") as f:
    pickle.dump(movies, f)
    

In [529]:
len(movies)

755

In [ ]:

"""
def release_dates(soup):
    try:
        node = soup.find(text=re.compile("Release dates"))
        node = node.parent.parent.findNextSibling()
        return [date.text for date in node.find_all("li") if len(date.text) > 0]
    except:
        return None

def budget(soup):
    try:
        node = soup.find(text=re.compile("Budget"))
        node = node.parent.findNextSibling()
        return node.text.split("[")[0]
    except:
        return None
    
def box_office(soup):
    try:
        node = soup.find(text=re.compile("Box office"))
        node = node.parent.findNextSibling()
        return node.text.split("[")[0]
    except:
        return None
    
        

for movie, data in movies.iteritems():
    print movie
    url = data['url']
    ## Go to each page and find the movie release date
    baseURL = "https://en.wikipedia.org/wiki/"
    response = requests.get(baseURL+url)
    soup = BeautifulSoup(response.text)
    movies[movie]['release dates'] = release_dates(soup)
    movies[movie]['budget'] = budget(soup)
  
    time.sleep(2) # Don't spam wikipedia TOOOO much....


with open("movies%d.pkl"%year, "wb") as f:
    pickle.dump(movies, f)

with open("movies%d.pkl"%year, "r") as f:
    movies = pickle.load(f)

for movie, data in movies.iteritems():
    print movie
    article = data['url']
    dates = data['release dates']
    dates = [parser.parse(date.split("(")[1].split(")")[0]) for date in data['release dates']]
    end_date = max(dates)
    start_date = min(dates) - timedelta(weeks = 12)
    
    
    unreleased = max(dates) > datetime.now() # Movie isn't out yet... can't do analysis
    nodata = (min(dates) - timedelta(weeks = 12)) < parser.parse("20150801") # Movie released before wikiViews existed
    already_saved = movie+".csv" in os.listdir("WikiVisits/") # already saved movie data!
    if unreleased or nodata or already_saved:
        continue

    url = ("https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/"
          "all-access/all-agents/%s/daily/%s/%s") % (article, 
                                                     str(start_date).split(" ")[0].replace("-", ""), 
                                                     str(end_date).split(" ")[0].replace("-", ""))

    response = requests.get(url)
    
    try:
        items = response.json()['items']
        views = [[item['timestamp'], item['views']] for item in items]

        with open("WikiVisits/"+movie+".csv", "w") as f:
            writer = csv.writer(f)
            writer.writerow(["timestamp", "views"])
            writer.writerows(views)
        print "Saved " + movie
    except:
        print movie + " failed"

    time.sleep(random.randint(2,5))"""

In [ ]:
def add_year(s):
    try:
        return s + "/%s"%str(year)[2:]
    except:
        return None

dfs = []
for year in [2014, 2015, 2016]:
    for page in range(1,8):
        df = pd.read_html("http://www.boxofficemojo.com/yearly/chart/?page=%d&view=releasedate&view2=domestic&yr=%s&p=.htm"%(page,year))
        df = df[3].iloc[5:, 1:8]
        
        df.columns = ["title", "studio", "gross", "total theaters", "opening gross", "opening theaters", "release"]
        df.release = df.release.apply(add_year)
        dfs.append(df)
        time.sleep(1)    
    
    print "Done:", year

In [572]:

df  = pd.concat(dfs, ignore_index=True) 

In [573]:
df

,title,studio,gross,total theaters,opening gross,opening theaters,release
0,American Sniper,WB,"$350,126,372",3885,"$633,456",4,12/25/14
1,The Hunger Games: Mockingjay - Part 1,LGF,"$337,135,885",4151,"$121,897,634",4151,11/21/14
2,Guardians of the Galaxy,BV,"$333,176,600",4088,"$94,320,883",4080,8/1/14
3,Captain America: The Winter Soldier,BV,"$259,766,572",3938,"$95,023,721",3938,4/4/14
4,The LEGO Movie,WB,"$257,760,692",3890,"$69,050,279",3775,2/7/14
5,The Hobbit: The Battle of the Five Armies,WB (NL),"$255,119,788",3875,"$54,724,334",3875,12/17/14
6,Transformers: Age of Extinction,Par.,"$245,439,076",4233,"$100,038,390",4233,6/27/14
7,Maleficent,BV,"$241,410,378",3948,"$69,431,298",3948,5/30/14
8,X-Men: Days of Future Past,Fox,"$233,921,534",4001,"$90,823,660",3996,5/23/14
9,Big Hero 6,BV,"$222,527,828",3773,"$56,215,889",3761,11/7/14


In [574]:
#df = pd.concat([pd.read_csv("finnancial2015.csv"), pd.read_csv("finnancial2016.csv")], ignore_index=True)
#df = df.iloc[:, 1:]
#finance = finance.set_index("title")
#finance.info()
#movie_list = os.listdir("WikiVisits/")
#df = finance.loc[[title.split(".")[0] for title in movie_list]].iloc[:, 1:].reset_index()
#df.head()
#df.info()

def strip_dollar_sign(s):
    try:
        return int(s.replace("$", "").replace(",",""))
    except (AttributeError, ValueError):
        return None
    
def to_datetime(t):
    try:
        return datetime.strptime(t, "%m/%d/%y")
    except:
        return None


df['opening gross'] = df['opening gross'].apply(strip_dollar_sign)
df['gross'] = df.gross.apply(strip_dollar_sign)
df.release = df.release.apply(to_datetime)
df['opening theaters'] = pd.to_numeric(df['opening theaters'], errors="coerce")
df['total theaters'] = pd.to_numeric(df['total theaters'], errors="coerce")

#df.sort_values("gross", ascending=False).head(10)
#df.append(pd.read_csv("finnancial.csv"), ignore_index=True)
#df.to_csv("finnancial.csv", encoding='utf-8', )


In [576]:
df.sample(10)

,title,studio,gross,total theaters,opening gross,opening theaters,release
1433,That Sugar Film,Gold.,3500,10,3500,10,2015-07-31
360,Mistaken for Strangers,Abr.,138991,9,87743,9,2014-03-28
1076,Fall in Love Like a Star,CL,212508,36,120074,36,2015-12-04
740,"Data as of: Today Today in 2014 Jan. 31, 20...","Data as of: Today Today in 2014 Jan. 31, 20...",NaN,NaN,NaN,NaN,NaT
455,The Discoverers,Quad,50020,4,7166,1,2014-05-16
96,The Best of Me,Rela.,26766213,2936,10003827,2936,2014-10-17
610,Marius,KL,9781,2,NaN,1,2014-07-04
1028,The Hunting Ground,RTWC,405917,25,22464,2,2015-02-27
116,Addicted,LGF,17390770,1037,7485346,846,2014-10-10
1147,Black Souls (Anime Nere),Alc,101088,11,6286,2,2015-04-10


In [577]:
len(movies)

755

In [582]:
wikiviews = {}
#wikiviews = pickle.load(open("wikiviews.pkl"))

In [580]:
df.sample(10)

,title,studio,gross,total theaters,opening gross,opening theaters,release
611,"If You Don't, I Will",FM,9719,1,1707,1,2014-12-17
693,The Hacker Wars,Vita.,3470,1,2958,1,2014-10-17
648,ABCs of Death 2,Magn.,7171,17,4022,17,2014-10-31
181,Bang Bang,FIP,2546172,271,1278690,271,2014-10-02
917,Youth,FoxS,2703296,149,78085,4,2015-12-04
1649,Neon Bull,KL,12490,4,6413,1,2016-04-08
265,Tracks (2014),Wein.,510007,67,21544,4,2014-09-19
309,Love on the Cloud (Wei Ai),CL,271734,17,67967,17,2014-12-24
333,Supermensch: The Legend of Shep Gordon,RTWC,213064,46,23017,4,2014-06-06
597,The Final Member,Drft.,11428,10,2064,7,2014-04-18


In [583]:
from collections import defaultdict
import json

movies = pickle.load(open("movies.pkl"))

for movie_title, series in df.set_index('title').loc[[title for title in df.title if title in movies]].iterrows():
    article_name = movies[movie_title]['url']
    
    if movie_title not in wikiviews:
        release_date = series.release.date()
        temp_dict = defaultdict(dict)
        for date in [release_date, release_date - timedelta(weeks=4)]:
            date = str(date.year)+"%02d" % date.month
            response = requests.get('http://stats.grok.se/json/en/%s/%s'%(date, urllib.quote(article_name)))
            if response.ok:
                response = response.json()
                temp_dict['daily_views'].update(response['daily_views'])
            else:
                print "Warning! Failed:", movie_title, date
                continue
                
        temp_dict['title'] = response['title']
        wikiviews[movie_title] = temp_dict

        with open("log.txt", "a") as log:
            json.dump(response, log)
        
        print movie_title
        
        time.sleep(.5)



American Sniper
Guardians of the Galaxy
Captain America: The Winter Soldier
The Hobbit: The Battle of the Five Armies
Transformers: Age of Extinction
Maleficent
X-Men: Days of Future Past
Big Hero 6
Dawn of the Planet of the Apes
The Amazing Spider-Man 2
22 Jump Street
Interstellar
How to Train Your Dragon 2
Gone Girl
Divergent
Neighbors
Ride Along
Rio 2
Into the Woods
Lucy
Unbroken
Night at the Museum: Secret of the Tomb
Mr. Peabody & Sherman
The Maze Runner
The Equalizer
Noah
Edge of Tomorrow
Non-Stop
The Imitation Game
Dumb and Dumber To
Tammy
Annabelle
Penguins of Madagascar
Let's Be Cops
The Monuments Men
The Purge: Anarchy
Alexander and the Terrible, Horrible, No Good, Very Bad Day
Think Like a Man Too
Exodus: Gods and Kings
The Nut Job
God's Not Dead
Son of God
The Grand Budapest Hotel
Planes: Fire & Rescue
Dracula Untold
Horrible Bosses 2
The Hundred-Foot Journey
Selma
Muppets Most Wanted
Ouija
The Boxtrolls
Jack Ryan: Shadow Recruit
If I Stay
The Judge
Jersey Boys
Blended
The 

In [585]:
wikiviews

{u"'71": defaultdict(dict,
             {'daily_views': {u'2015-01-01': 1024,
               u'2015-01-02': 1365,
               u'2015-01-03': 1179,
               u'2015-01-04': 1325,
               u'2015-01-05': 1176,
               u'2015-01-06': 1264,
               u'2015-01-07': 1239,
               u'2015-01-08': 1014,
               u'2015-01-09': 1626,
               u'2015-01-10': 1414,
               u'2015-01-11': 1264,
               u'2015-01-12': 1127,
               u'2015-01-13': 942,
               u'2015-01-14': 852,
               u'2015-01-15': 934,
               u'2015-01-16': 901,
               u'2015-01-17': 808,
               u'2015-01-18': 909,
               u'2015-01-19': 981,
               u'2015-01-20': 945,
               u'2015-01-21': 693,
               u'2015-01-22': 719,
               u'2015-01-23': 1427,
               u'2015-01-24': 936,
               u'2015-01-25': 1089,
               u'2015-01-26': 1009,
               u'2015-01-27': 840

In [586]:
pickle.dump(wikiviews, open("wikiviews.pkl", "w"))

In [587]:
df = df.set_index("title")
def to_datetime(t):
    try:
        return datetime.strptime(t, '%Y-%m-%d')
    except ValueError:
        return pd.NaT
    
df['wiki views'] = np.NaN
for movie in wikiviews.keys():
    views = pd.DataFrame(wikiviews[movie]['daily_views'].items(), columns=['time', 'views'])
    
    if len(views) == 0:
        print "Warning: %s has no data" % movie
        continue
        
    views.time = views.time.apply(to_datetime)
    
    views.sort_values("time")
    df.loc[movie, 'wiki views'] = views[views.time > (views.time.iloc[-1] - timedelta(weeks=4))].views.sum()


In [594]:
df.to_csv("masterDF.csv", encoding='utf-8')

In [593]:
df

,studio,gross,total theaters,opening gross,opening theaters,release,wiki views
title,,,,,,,
American Sniper,WB,350126372,3885,633456,4,2014-12-25,66393
The Hunger Games: Mockingjay - Part 1,LGF,337135885,4151,121897634,4151,2014-11-21,NaN
Guardians of the Galaxy,BV,333176600,4088,94320883,4080,2014-08-01,3559766
Captain America: The Winter Soldier,BV,259766572,3938,95023721,3938,2014-04-04,3104256
The LEGO Movie,WB,257760692,3890,69050279,3775,2014-02-07,NaN
The Hobbit: The Battle of the Five Armies,WB (NL),255119788,3875,54724334,3875,2014-12-17,1472946
Transformers: Age of Extinction,Par.,245439076,4233,100038390,4233,2014-06-27,2172606
Maleficent,BV,241410378,3948,69431298,3948,2014-05-30,779471
X-Men: Days of Future Past,Fox,233921534,4001,90823660,3996,2014-05-23,3099755


In [547]:
df

,title,studio,gross,total theaters,opening gross,opening theaters,release
0,American Sniper,WB,350126372,3885,633456,4,2016-12-25
1,The Hunger Games: Mockingjay - Part 1,LGF,337135885,4151,121897634,4151,2016-11-21
2,Guardians of the Galaxy,BV,333176600,4088,94320883,4080,2016-08-01
3,Captain America: The Winter Soldier,BV,259766572,3938,95023721,3938,2016-04-04
4,The LEGO Movie,WB,257760692,3890,69050279,3775,2016-02-07
5,The Hobbit: The Battle of the Five Armies,WB (NL),255119788,3875,54724334,3875,2016-12-17
6,Transformers: Age of Extinction,Par.,245439076,4233,100038390,4233,2016-06-27
7,Maleficent,BV,241410378,3948,69431298,3948,2016-05-30
8,X-Men: Days of Future Past,Fox,233921534,4001,90823660,3996,2016-05-23
9,Big Hero 6,BV,222527828,3773,56215889,3761,2016-11-07


In [596]:
import re 
url = 'https://en.wikipedia.org/wiki/Quentin_Tarantino'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")

/Library/Python/2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
